In [5]:
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, r2_score, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# --- Upload the CSV file ---
uploaded = files.upload()

# --- Load dataset ---
df = pd.read_csv("t20i_Matches_Data_final.csv")

# --- Helper function to preprocess data ---
def preprocess_data(df, target_col, is_classification=True):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = np.expand_dims(X_scaled, axis=2)  # (samples, features, 1)

    if is_classification:
        # Encode target labels (handles non-sequential integers)
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)
        num_classes = len(np.unique(y_encoded))
        y_encoded = to_categorical(y_encoded, num_classes)
        return X_scaled, y_encoded, num_classes
    else:
        y = y.astype(float)
        return X_scaled, y, None

# --- Helper function to build a 1D CNN ---
def build_cnn(input_shape, num_classes=None, is_classification=True):
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(Flatten())
    model.add(Dropout(0.3))
    if is_classification:
        model.add(Dense(num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    else:
        model.add(Dense(1, activation='linear'))
        model.compile(optimizer='adam', loss='mse')
    return model


# ========== CNN CLASSIFIER 1: Match Winner ==========
X1, y1, n_classes1 = preprocess_data(df, "Match Winner", is_classification=True)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

model1 = build_cnn(X1_train.shape[1:], n_classes1, is_classification=True)
model1.fit(X1_train, y1_train, epochs=25, batch_size=32, verbose=0)

y1_pred = np.argmax(model1.predict(X1_test), axis=1)
y1_true = np.argmax(y1_test, axis=1)

print("\n📊 CNN Classifier (Match Winner)")
print("Accuracy:", accuracy_score(y1_true, y1_pred))
print("Precision:", precision_score(y1_true, y1_pred, average='weighted'))
print("Recall:", recall_score(y1_true, y1_pred, average='weighted'))

# Save model
model1.save("cnn_match_winner.h5")
print("Model saved as cnn_match_winner.h5")


# ========== CNN CLASSIFIER 2: Toss Winner ==========
X2, y2, n_classes2 = preprocess_data(df, "Toss Winner", is_classification=True)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

model2 = build_cnn(X2_train.shape[1:], n_classes2, is_classification=True)
model2.fit(X2_train, y2_train, epochs=25, batch_size=32, verbose=0)

y2_pred = np.argmax(model2.predict(X2_test), axis=1)
y2_true = np.argmax(y2_test, axis=1)

print("\n📊 CNN Classifier (Toss Winner)")
print("Accuracy:", accuracy_score(y2_true, y2_pred))
print("Precision:", precision_score(y2_true, y2_pred, average='weighted'))
print("Recall:", recall_score(y2_true, y2_pred, average='weighted'))

# Save model
model2.save("cnn_toss_winner.h5")
print("Model saved as cnn_toss_winner.h5")


# ========== CNN REGRESSOR: Toss Winner Choice_bat ==========
X3, y3, _ = preprocess_data(df, "Toss Winner Choice_bat", is_classification=False)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=42)

model3 = build_cnn(X3_train.shape[1:], is_classification=False)
model3.fit(X3_train, y3_train, epochs=25, batch_size=32, verbose=0)

y3_pred = model3.predict(X3_test).flatten()

print("\n📈 CNN Regressor (Toss Winner Choice_bat)")
print("R² Score:", r2_score(y3_test, y3_pred))
print("MSE:", mean_squared_error(y3_test, y3_pred))

# Save model
model3.save("cnn_toss_choice_bat_regressor.h5")
print("Model saved as cnn_toss_choice_bat_regressor.h5")


Saving t20i_Matches_Data_final.csv to t20i_Matches_Data_final (3).csv


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



📊 CNN Classifier (Match Winner)
Accuracy: 0.24084778420038536
Precision: 0.23610929990668034
Recall: 0.24084778420038536
Model saved as cnn_match_winner.h5


...

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



📊 CNN Classifier (Toss Winner)
Accuracy: 0.1676300578034682
Precision: 0.15395033052377796
Recall: 0.1676300578034682
Model saved as cnn_toss_winner.h5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step



📈 CNN Regressor (Toss Winner Choice_bat)
R² Score: 0.9535109833976771
MSE: 0.011563185221032102
Model saved as cnn_toss_choice_bat_regressor.h5
